Created by [Nathan Kelber](http://nkelber.com) and Ted Lawless for [JSTOR Labs](https://labs.jstor.org/) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
**For questions/comments/improvements, email nathan.kelber@ithaka.org.**<br />
![CC BY License Logo](https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png)
___

# Latent Dirichlet Allocation (LDA) Topic Modeling

**Description of methods in this notebook:**
This [notebook](https://docs.tdm-pilot.org/key-terms/#jupyter-notebook) demonstrates how to do topic modeling on a [JSTOR](https://docs.tdm-pilot.org/key-terms/#jstor) and/or [Portico](https://docs.tdm-pilot.org/key-terms/#portico) [dataset](https://docs.tdm-pilot.org/key-terms/#dataset) using [Python](https://docs.tdm-pilot.org/key-terms/#python). The following processes are described:

* Importing your [dataset](https://docs.tdm-pilot.org/key-terms/#dataset)
* Importing libraries including `gensim`, `nltk`, and `pyLDAvis`
* Writing a helper function to help clean up a single [token](https://docs.tdm-pilot.org/key-terms/#token)
* Building a gensim dictionary and training the model
* Computing a topic list
* Visualizing the topic list

**Difficulty:** Intermediate

**Knowledge Required:** 
* [Python Basics I](./0-python-basics-1.ipynb)
* [Python Basics II](./0-python-basics-2.ipynb)
* [Python Basics III](./0-python-basics-3.ipynb)

**Knowledge Recommended:**
* [Exploring Metadata](./1-metadata.ipynb)
* A familiarity with [gensim](https://docs.tdm-pilot.org/key-terms/#gensim) is helpful but not required.

**Completion time:** 90 minutes

**Data Format:** [JSTOR](https://docs.tdm-pilot.org/key-terms/#jstor)/[Portico](https://docs.tdm-pilot.org/key-terms/#portico) [JSON Lines (.jsonl)](https://docs.tdm-pilot.org/key-terms/#jsonl)

**Libraries Used:**
* **[json](https://docs.tdm-pilot.org/key-terms/#json-python-library)** to convert our dataset from json lines format to a Python list
**[gensim](https://docs.tdm-pilot.org/key-terms/#gensim)** to accomplish the topic modeling
**pyldavis** to visualize our topic model
____










This example uses the [`gensim`](https://radimrehurek.com/gensim/index.html) library for building the topic model. A familiarity with gensim is helpful but not required.
____

## Import your dataset

You'll use the tdm_client library to automatically upload your dataset. We import the `Dataset` module from the `tdm_client` library. The tdm_client library contains functions for connecting to the JSTOR server containing our [corpus](https://docs.tdm-pilot.org/key-terms/#corpus) [dataset](https://docs.tdm-pilot.org/key-terms/#dataset). To analyze your dataset, use the [dataset ID](https://docs.tdm-pilot.org/key-terms//#dataset-ID) provided when you created your [dataset](https://docs.tdm-pilot.org/key-terms//#dataset). A copy of your [dataset ID](https://docs.tdm-pilot.org/key-terms//#dataset-ID) was sent to your email when you created your [corpus](https://docs.tdm-pilot.org/key-terms/#corpus). It should look like a long series of characters surrounded by dashes. If you haven't created a dataset, feel free to use a sample dataset. Here's a [list by discipline](https://docs.tdm-pilot.org/sample-datasets/). Advanced users can also [upload a dataset from their local machine](https://docs.tdm-pilot.org/uploading-a-dataset/).

In [1]:
import json

In [2]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import gensim

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

/Users/nkelber/opt/anaconda3/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/Users/nkelber/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/nkelber/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, defaultdict
/Users/nkelber/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/vocabulary.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' 

In [3]:
#Importing your dataset with a dataset ID
import tdm_client
#Load the sample dataset, the full run of Shakespeare Quarterly from 1950-2013.
tdm_client.get_dataset("f6ae29d4-3a70-36ee-d601-20a8c0311273", "sampleJournalAnalysis") 

'datasets/sampleJournalAnalysis.jsonl'

Initialize a dataset object. 

Define a function for processing tokens from the extracted features for volumes in the curated dataset. This function:

* lowercases all tokens
* discards all tokens less than 4 characters
* discards non alphabetical tokens - e.g. --9
* removes stopwords using NLTK's stopword list
* Lemmatizes the token using NLTK's [WordNetLemmatizer](https://www.nltk.org/_modules/nltk/stem/wordnet.html)

In [4]:

def process_token(token):
    token = token.lower()
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    if token in stop_words:
        return
    return WordNetLemmatizer().lemmatize(token)

Loop through the documents in the dataset and build a list of doucments where each document is a list of tokens. For demonstration purposes, we are going to limit to 25 documents in the dataset. 

In [5]:
documents = []
doc_count = 0
# Limit the number of documents, set to None to not limit.
limit_to = 25

with open("./datasets/sampleJournalAnalysis.jsonl") as input_file:
    for line in input_file:
        doc = json.loads(line)
        unigram_count = doc["unigramCount"]
        document_tokens = []
        for token, count in unigram_count.items():
            clean_token = process_token(token)
            if clean_token is None:
                continue
            document_tokens += [clean_token] * count
        documents.append(document_tokens)
        doc_count += 1 
        if (limit_to is not None) and (doc_count >= limit_to):
            break
                

Build a gensim dictionary corpus and then train the model.

In [6]:

num_topics = 10

dictionary = gensim.corpora.Dictionary(documents)

# Remove terms that appear in less than 10% of documents and more than 75% of documents.
dictionary.filter_extremes(no_below=doc_count * .10, no_above=0.75)

bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

# Train the LDA model.
model = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=15
)


Print the most significant terms, as determined by the model, for each topic.

In [7]:
for topic_num in range(0, num_topics):
    word_ids = model.get_topic_terms(topic_num)
    words = []
    for wid, weight in word_ids:
        word = dictionary.id2token[wid]
        words.append(word)
    print("Topic {}".format(str(topic_num).ljust(5)), " ".join(words))

Topic 0     book critical criticism tragedy essay complete four title excerpt character
Topic 1     book work make male woman view critical character female question
Topic 2     william john sonnet well manuscript found many became private copy
Topic 3     portrait macbeth bassanio shylock merchant money original exchange shall must
Topic 4     company james gentleman stage production romeo michael david spirit museum
Topic 5     macbeth production work original made lady stage like portrait theatre
Topic 6     macbeth lear like stage king shylock make line work production
Topic 7     stage john like richard production henry much made theatre king
Topic 8     lear king edgar form report like meaning worst date creative
Topic 9     troilus public version quarto private kenneth robert oxford secret theatre


Visualize the model using [`pyLDAvis`](https://pyldavis.readthedocs.io/en/latest/). This visualization takes several minutes to an hour to generate depending on the size of your dataset. To run, remove the `#` symbol on the line below and run the cell. 

In [8]:
pyLDAvis.gensim.prepare(model, bow_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.046652  0.014911       1        1  39.443951
3      0.066325 -0.048510       2        1  25.582691
1      0.079981  0.107990       3        1  16.628067
8      0.183318  0.031184       4        1  13.170895
4     -0.153466  0.097143       5        1   1.764015
9     -0.044155 -0.071969       6        1   1.630523
2     -0.010235 -0.126808       7        1   1.141054
0     -0.031109  0.010456       8        1   0.607107
5     -0.021763 -0.006769       9        1   0.015850
6     -0.022245 -0.007627      10        1   0.015850, topic_info=          Term       Freq      Total Category  logprob  loglift
587    william  50.000000  50.000000  Default  30.0000  30.0000
264       john  78.000000  78.000000  Default  29.0000  29.0000
12        book  54.000000  54.000000  Default  28.0000  28.0000
268       lear  81.000000  81.000000  Default  27.0000  27.0000
140      stage  68.000000  68.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
112  presented   0.001579  35.369469  Topic10  -7.3645  -1.2670
587    william   0.001584  50.290733  Topic10  -7.3612  -1.6156
651       life   0.001581  42.026703  Topic10  -7.3632  -1.4381
634       form   0.001580  44.448570  Topic10  -7.3642  -1.4951
279       mean   0.001579  39.933468  Topic10  -7.3644  -1.3882

[668 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
164       4  0.874740  abandoned
781       2  0.204334   accepted
781       4  0.613003   accepted
781       7  0.204334   accepted
1505      1  0.631656   achieved
...     ...       ...        ...
589       4  0.059130       year
589       5  0.059130       year
1133      2  0.331250    younger
1133      3  0.331250    younger
1133      7  0.331250    younger

[1277 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 4, 2, 9, 5, 10, 3, 1, 6, 7])